In [ ]:
# Permutation table S
S = [
   41, 46, 67, 201, 162, 216, 124, 1, 61, 54, 84, 161, 236, 240, 6,
  19, 98, 167, 5, 243, 192, 199, 115, 140, 152, 147, 43, 217, 188,
  76, 130, 202, 30, 155, 87, 60, 253, 212, 224, 22, 103, 66, 111, 24,
  138, 23, 229, 18, 190, 78, 196, 214, 218, 158, 222, 73, 160, 251,
  245, 142, 187, 47, 238, 122, 169, 104, 121, 145, 21, 178, 7, 63,
  148, 194, 16, 137, 11, 34, 95, 33, 128, 127, 93, 154, 90, 144, 50,
  39, 53, 62, 204, 231, 191, 247, 151, 3, 255, 25, 48, 179, 72, 165,
  181, 209, 215, 94, 146, 42, 172, 86, 170, 198, 79, 184, 56, 210,
  150, 164, 125, 182, 118, 252, 107, 226, 156, 116, 4, 241, 69, 157,
  112, 89, 100, 113, 135, 32, 134, 91, 207, 101, 230, 45, 168, 2, 27,
  96, 37, 173, 174, 176, 185, 246, 28, 70, 97, 105, 52, 64, 126, 15,
  85, 71, 163, 35, 221, 81, 175, 58, 195, 92, 249, 206, 186, 197,
  234, 38, 44, 83, 13, 110, 133, 40, 132, 9, 211, 223, 205, 244, 65,
  129, 77, 82, 106, 220, 55, 200, 108, 193, 171, 250, 36, 225, 123,
  8, 12, 189, 177, 74, 120, 136, 149, 139, 227, 99, 232, 109, 233,
  203, 213, 254, 59, 0, 29, 57, 242, 239, 183, 14, 102, 88, 208, 228,
  166, 119, 114, 248, 235, 117, 75, 10, 49, 68, 80, 180, 143, 237,
  31, 26, 219, 153, 141, 51, 159, 17, 131, 20
]

def md2(data: bytes) -> bytes:
    # Pad to multiple of 16 bytes
    pad_len = 16 - (len(data) % 16)
    data += bytes([pad_len]) * pad_len

    # Compute 16‐byte checksum
    L = 0
    checksum = bytearray(16)
    for i in range(0, len(data)//16):
        for j in range(16):
            c = data[i*16 +j]
            # checksum[j]=S[c ^ L]
            # L= checksum[j]
            L = checksum[j] ^ S[c ^ L]
            checksum[j] = L
    data += bytes(checksum)

    # Initialize 48‐byte state X
    X = [0]*48

    #  Process each 16‐byte block
    for i in range(0, len(data) // 16):
        for j in range(16):
            X[16+j] = data[i * 16 + j]
            X[32+j] = X[j] ^ X[16+j]

        # 18 rounds of permutation
        t = 0
        for round in range(18):
            for k in range(48):
                t = X[k] ^ S[t]
                X[k] = t
            t = (t + round) & 0xFF

    #  Output first 16 bytes of state
    return bytes(X[:16])


if __name__ == "__main__":
    tests = {
        b"":           "8350e5a3e24c153df2275c9f80692773",
        b"a":          "32ec01ec4a6dac72c0ab96fb34c0b5d1",
        b"abc":        "da853b0d3f88d99b30283a69e6ded6bb",
        b"message digest": "ab4f496bfb2a530b219ff33031fe06b0",
        b"abcdefghijklmnopqrstuvwxyz": "4e8ddff3650292ab5a4108c3aa47940b",
    }

    for msg, expected in tests.items():
        out = md2(msg).hex()
        print(f"{msg!r}: {out}  {'OK' if out==expected else 'FAIL (got '+out+')'}")


b'': 8350e5a3e24c153df2275c9f80692773  OK
b'a': 32ec01ec4a6dac72c0ab96fb34c0b5d1  OK
b'abc': da853b0d3f88d99b30283a69e6ded6bb  OK
b'message digest': ab4f496bfb2a530b219ff33031fe06b0  OK
b'abcdefghijklmnopqrstuvwxyz': 4e8ddff3650292ab5a4108c3aa47940b  OK


In [11]:
from Crypto.Hash import MD2
import random
words = [
    "sun", "moon", "sky", "river", "tree", "earth", "light", "shadow", "star", "fire",
    "stone", "glass", "sand", "mountain", "cloud", "storm", "rain", "snow", "ice", "ocean",
    "wind", "desert", "leaf", "bark", "root", "branch", "petal", "flame", "ash", "echo",
    "prasanna","healing"  , "pkskakdsjcscocjsdc0ci" ,"2oe20d0c0cdsd0cad0c d0ak dk " ," dkskcskc  5w25@!!@33"
 ]

samples = random.sample(words, 25)

print("\nVerifying against Crypto.Hash.MD2:")
for word in samples:
    my_hash = md2(word.encode()).hex()

    h = MD2.new()
    h.update(word.encode())
    crypto_hash = h.hexdigest()

    status = "✔️ OK" if my_hash == crypto_hash else f"❌ FAIL (got {my_hash})"
    print(f"{word!r}: {crypto_hash} - {status}")


Verifying against Crypto.Hash.MD2:
'stone': f048d92dc561e9db7e9e0b8db056418c - ✔️ OK
'sky': 252f4a5206df02bce3be027e8944dd7e - ✔️ OK
'tree': 9dcd1ee6f6c3eeeb6971475f0f6ecd9d - ✔️ OK
'healing': eed44e6e6554166626786704eb64284c - ✔️ OK
'mountain': 8082e5c3ec54d64a6b6c0ae898e9e95a - ✔️ OK
'prasanna': c10b566ba35fcd5255fa56dfe1d20314 - ✔️ OK
'sun': 52550b80f7cf7726926a5b8be66404d5 - ✔️ OK
'desert': 800631363cd0cfcb950b387dbe0795b5 - ✔️ OK
'branch': 30c368734a9f4d3bef7c906d81bf1747 - ✔️ OK
'moon': 91aa10590874c834fb4b08fbbaf24ac6 - ✔️ OK
'star': 8bd0bd8cd79b2c5569205e2b8f13d4a1 - ✔️ OK
'rain': f202834d19387fe79d3b4843f752b750 - ✔️ OK
'ash': 97a9c068c7fe4b85286f6d290af68463 - ✔️ OK
'pkskakdsjcscocjsdc0ci': fea9280003098e44faee0ded997fc697 - ✔️ OK
'flame': 8ba222fdefac9b4254e25c32153e0f53 - ✔️ OK
'echo': 273c54f716150ffd9dcd11671e3dc823 - ✔️ OK
'fire': 7b0eacc9164c3e63ba470262763bc771 - ✔️ OK
'wind': fcc50c368f63fa070de9e35f2e06f241 - ✔️ OK
'earth': 2a8a6c645ddf5d628330b3591950ab97 - ✔️ OK
'